In [1]:
!pip install fastapi uvicorn pyngrok transformers torch pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 64.4 MB/s eta 0:00:00:00:0100:01


In [2]:
!pip install python-multipart

In [ ]:
from fastapi import FastAPI, UploadFile, File, HTTPException
from pydantic import BaseModel
from fastapi.middleware.cors import CORSMiddleware
import fitz  # PyMuPDF for extracting text
from transformers import pipeline
import os
from pyngrok import ngrok
import uvicorn
import nest_asyncio

# Apply nest_asyncio to enable the event loop within Kaggle/Colab
nest_asyncio.apply()

# Initialize FastAPI app
app = FastAPI()

# Enable CORS for frontend access
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Update for specific domains in production
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# In-memory storage for documents (Use a database in production)
documents = {}

# Load a QA model for question answering
qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")

class QuestionRequest(BaseModel):
    doc_id: int
    question: str

@app.post("/upload/")
async def upload_pdf(file: UploadFile = File(...)):
    # Validate file type
    if file.content_type != "application/pdf":
        raise HTTPException(status_code=400, detail="Only PDF files are supported.")

    # Create the uploads directory if it doesn't exist
    os.makedirs("uploads", exist_ok=True)
    file_path = f"uploads/{file.filename}"

    # Save the uploaded file
    try:
        with open(file_path, "wb") as f:
            f.write(file.file.read())
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Failed to save file: {str(e)}")

    # Extract text from the PDF
    try:
        doc_text = extract_text_from_pdf(file_path)
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Failed to extract text: {str(e)}")

    # Assign a document ID and store the text
    doc_id = len(documents) + 1
    documents[doc_id] = {"filename": file.filename, "text": doc_text}

    return {"doc_id": doc_id, "filename": file.filename}

@app.post("/ask/")
async def ask_question(request: QuestionRequest):
    # Retrieve the document by ID
    doc = documents.get(request.doc_id)
    if not doc:
        raise HTTPException(status_code=404, detail="Document not found.")

    # Generate an answer using the QA pipeline
    try:
        answer = generate_answer(request.question, doc["text"])
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Failed to generate answer: {str(e)}")

    return {"answer": answer}

def extract_text_from_pdf(file_path: str) -> str:
    """Extract text from each page of the PDF file."""
    text = ""
    try:
        with fitz.open(file_path) as doc:
            for page in doc:
                text += page.get_text()
    except Exception as e:
        raise Exception(f"Error reading PDF file: {str(e)}")

    return text.strip()

def generate_answer(question: str, doc_text: str) -> str:
    """Generate a concise answer from the document text using a free model."""
    max_context_length = 1000  # Limit the context to 1000 characters for simplicity
    context = doc_text[:max_context_length]

    try:
        result = qa_pipeline({
            "question": question,
            "context": context
        })
        return result.get("answer", "No answer found.")
    except Exception as e:
        raise Exception(f"Error during QA pipeline execution: {str(e)}")

# Set up ngrok tunnel for public access
ngrok.set_auth_token("2oI5k7GAl2VUhiMosfoP2L2RTMr_sTh4Rx4fJRDHpMz9Zz1R")  # Replace with your ngrok token
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

# Run the FastAPI app with Uvicorn
uvicorn.run(app, host="0.0.0.0", port=8000)


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Public URL: NgrokTunnel: "https://c47a-35-223-16-116.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [30]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     203.129.246.110:0 - "POST /upload/ HTTP/1.1" 400 Bad Request
INFO:     203.129.246.110:0 - "POST /upload/ HTTP/1.1" 422 Unprocessable Entity
INFO:     203.129.246.110:0 - "POST /upload/ HTTP/1.1" 400 Bad Request
INFO:     203.129.246.110:0 - "POST /upload/ HTTP/1.1" 200 OK
INFO:     203.129.246.110:0 - "POST /ask/ HTTP/1.1" 200 OK
INFO:     203.129.246.110:0 - "POST /upload/ HTTP/1.1" 200 OK
INFO:     203.129.246.110:0 - "OPTIONS /ask/ HTTP/1.1" 200 OK
INFO:     203.129.246.110:0 - "POST /ask/ HTTP/1.1" 200 OK
